In [ ]:
"""
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


# Get a batch of training data
#inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
#out = torchvision.utils.make_grid(inputs)

#imshow(out, title=[class_names[x] for x in classes])

def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title(f'predicted: {class_names[preds[j]]}')
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)
"""

import cv2
import matplotlib.pyplot as plt
import os

### 绘制三个图片
def plot_3_img(img1, img2, img3):
    bGray = (len(img1.shape) == 2)
    plt.figure(figsize=(12, 3))
    ax1 = plt.subplot(131)
    ax2 = plt.subplot(132)
    ax3 = plt.subplot(133)
    if bGray:
        ax1.imshow(img1, cmap='gray')
        ax2.imshow(img2, cmap='gray')
        ax3.imshow(img3, cmap='gray')
    else:
        ax1.imshow(img1)
        ax2.imshow(img2)s
        ax3.imshow(img3)

In [ ]:
# model_ft = models.resnet50(weights="IMAGENET1K_V2")
# for param in model_ft.parameters():
#     param.requires_grad = False
# num_ftrs = model_ft.fc.in_features

#model_ft = torchvision.models.quantization.mobilenet_v3_large(weights="IMAGENET1K_V2")
#for param in model_ft.parameters():
#    param.requires_grad = False
#num_ftrs = model_ft.classifier[3].in_features

# model_ft = torchvision.models.mobilenet_v3_small(pretrained=True)
# num_ftrs = model_ft.classifier[3].in_features

#model_ft = torchvision.models.mobilenet_v3_large(pretrained=True)
#num_ftrs = model_ft.classifier[3].in_features

# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to ``nn.Linear(num_ftrs, len(class_names))``.
# model_ft.fc = nn.Linear(num_ftrs, 2)

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory
from MyImageDataset import MyImageDataset

cudnn.benchmark = True
plt.ion()   # interactive mode

# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        #transforms.ColorJitter(brightness=[0.8, 1.5], contrast=[0.8, 1.2], saturation=[0.8, 1.2], hue=(0.3, 0.5))
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = r'D:/pictures/wydm/svm'


image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4)  for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def train_model(model, criterion, optimizer, scheduler, num_epochs=25, model_name="model.pt"):
    since = time.time()

    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(r"C:\Users\LEGION", model_name)

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in dataloaders[phase]:
                    inputs = inputs.to(device)
                    labels = labels.to(device)

                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # forward
                    # track history if only in train
                    with torch.set_grad_enabled(phase == 'train'):
                        outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss = criterion(outputs, labels)

                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss.backward()
                            optimizer.step()

                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights
        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [34]:
# model_ft = models.efficientnet_v2_m(weights="IMAGENET1K_V1")
# print(model_ft.classifier)

# for param in model_ft.parameters():
#     param.requires_grad = False
# num_ftrs = model_ft.classifier[1].in_features
# model_ft.classifier[1] = nn.Linear(num_ftrs, 2)

# model_ft = models.vit_b_16(weights="IMAGENET1K_SWAG_LINEAR_V1")
# print(model_ft.heads)

# model_ft = models.resnet50(weights="IMAGENET1K_V2")
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)

model_ft = torchvision.models.mobilenet_v3_small(pretrained=True)
num_ftrs = model_ft.classifier[3].in_features
model_ft.classifier[3] = nn.Linear(num_ftrs, 2)

# model_ft = models.shufflenet_v2_x0_5(weights="IMAGENET1K_V1")
# num_ftrs = model_ft.fc.in_features
# model_ft.fc = nn.Linear(num_ftrs, 2)

print(model_ft)


model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.8)

MobileNetV3(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      (2): Hardswish()
    )
    (1): InvertedResidual(
      (block): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
          (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
        (1): SqueezeExcitation(
          (avgpool): AdaptiveAvgPool2d(output_size=1)
          (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
          (fc2): Conv2d(8, 16, kernel_size=(1, 1), stride=(1, 1))
          (activation): ReLU()
          (scale_activation): Hardsigmoid()
        )
        (2): Conv2dNormActivation(
          (0): Conv2d(16, 16, kernel_size=(1, 1), 

### 训练

In [9]:
# 0.928  Res50   4~6ms
# 0.940653 MobileNetV3_large Quantization 6~10ms
# 0.925816 MobileNetV3_small 4ms
# 0.937685 MobileNetV3_large 8ms~11ms

# 0.915 Res50
# 0.9263 MobileNetV3_large Quantization
# Res50 冻结卷积层，0.863939
# 0.928900 efficientnet_v2_m
# efficientnet_v2_m 增加数据，修正几条数据:  0.936159

# efficientnet_v2_m 增加数据，修正数据，重新划分数据集： 0.9194

# WYDM_efficientnet_v2_m_6000_20230909_4.pt： 0.9220
# WYDM_efficientnet_v2_m_6000_20230909_5.pt： 0.9263
# WYDM_efficientnet_v2_m_6000_20230909_6.pt： 0.9298
# WYDM_efficientnet_v2_m_6000_20230909_6.pt： 0.933124
# WYDM_efficientnet_v2_m_6000_20230909_7.pt： 0.9434
# WYDM_efficientnet_v2_m_6000_20230909_8.pt： 0.928571
# WYDM_efficientnet_v2_m_6000_20230909_9.pt： 0.942464
# WYDM_efficientnet_v2_m_6000_20230909_10.pt：0.942152
# WYDM_efficientnet_v2_m_6000_20230909_11.pt：0.956575
# WYDM_efficientnet_v2_m_6000_20230909_12.pt：0.948780
# WYDM_efficientnet_v2_m_6000_20230909_13.pt：0.948655
# WYDM_efficientnet_v2_m_6000_20230909_14.pt：0.957120
# WYDM_efficientnet_v2_m_6000_20230909_15.pt：0.955170

# WYDM_efficientnet_v2_m_20230905_1.pt：0.956785

# 增加模型复杂度后
# WYDM_efficientnet_v2_m_20231003_1.pt：0.959175
# WYDM_efficientnet_v2_m_20231003_2.pt：0.961398
# WYDM_efficientnet_v2_m_20231003_3.pt：0.956645
# WYDM_efficientnet_v2_m_20231003_4.pt：0.959789

# Transformer模型
# vit_20231004_1.pt：0.8371

# Resnet 50
# Resnet50_20231005_1.pt：0.947604

# regnet
# regnet_20231005_1.pt：内存太小

# shufflenet
# shufflenet_20231005_1.pt：0.902


### 对烟丝图片训练结果
# Resnet50_20231008_1.pt：0.928571
# mobilenet_v3_small_20231008_1.pt：0.924370
# shufflenet_20231008_1.pt：0.869748


model_name = "shufflenet_20231008_1.pt"

model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=15, model_name=model_name)

Epoch 0/14
----------
train Loss: 0.6921 Acc: 0.5095
val Loss: 0.6888 Acc: 0.5504
Epoch 1/14
----------
train Loss: 0.6885 Acc: 0.5725
val Loss: 0.6847 Acc: 0.8109
Epoch 2/14
----------
train Loss: 0.6852 Acc: 0.7248
val Loss: 0.6803 Acc: 0.8319
Epoch 3/14
----------
train Loss: 0.6817 Acc: 0.6933
val Loss: 0.6776 Acc: 0.8403
Epoch 4/14
----------
train Loss: 0.6794 Acc: 0.7321
val Loss: 0.6741 Acc: 0.8571
Epoch 5/14
----------
train Loss: 0.6766 Acc: 0.7868
val Loss: 0.6708 Acc: 0.8697
Epoch 6/14
----------
train Loss: 0.6740 Acc: 0.7857
val Loss: 0.6684 Acc: 0.8319
Epoch 7/14
----------
train Loss: 0.6713 Acc: 0.7889
val Loss: 0.6660 Acc: 0.8487
Epoch 8/14
----------
train Loss: 0.6722 Acc: 0.7742
val Loss: 0.6644 Acc: 0.8487
Epoch 9/14
----------
train Loss: 0.6683 Acc: 0.8025
val Loss: 0.6627 Acc: 0.8529
Epoch 10/14
----------
train Loss: 0.6682 Acc: 0.7952
val Loss: 0.6602 Acc: 0.8655
Epoch 11/14
----------
train Loss: 0.6673 Acc: 0.7910
val Loss: 0.6605 Acc: 0.8613
Epoch 12/14
--

### pytorch转onnx

In [31]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = "mobilenet_v3_small_20231008_1.pt"
onnx_name = "mobilenet_v3_small_20231008_1.onnx"
model_ft.load_state_dict(torch.load(model_name))
model_ft.eval()


# with torch.autocast("cuda", dtype=torch.float16):
#     inputs = torch.randn(1, 3, 224, 224, device="cuda")
inputs = torch.randn(1, 3, 224, 224)
inputs = inputs.to(device)
# torch.set_default_tensor_type('torch.FloatTensor')
# torch.set_default_tensor_type('torch.cuda.FloatTensor')

torch.onnx.export(
    model_ft,
    inputs,
    "D:/models/" + onnx_name,
    export_params=True,
    opset_version=14,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    #     dynamic_axes={"input": {0: "batch_size", 1: "channel", 2: "height", 3: "width"}},
)    

============= Diagnostic Run torch.onnx.export version 2.0.1+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [2]:
import time
import onnxruntime
onnx_name = "mobilenet_v3_small_20231008_1.onnx"
onnx_model = onnxruntime.InferenceSession("D:/models/" + onnx_name, providers=['CUDAExecutionProvider'])
# output_name = []
# for node in onnx_model.get_outputs():
#     output_name.append(node.name)
# print(output_name)
# onnx_outputs = onnx_model.run(output_name, input_feed={"input": img})

image_datasets = {x: MyImageDataset(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1, shuffle=False, num_workers=4)  for x in ['train', 'val']}

t1 = time.time()
bad_ct = 0
bad_files = []
for inputs, labels, file_paths in dataloaders['val']:
#     inputs = inputs.to(device)
    inputs = inputs.numpy()
    labels = labels.numpy()
    onnx_outputs = onnx_model.run(['output'], input_feed={"input": inputs})
    if np.argmax(onnx_outputs[0][0]) != labels[0]:
        bad_ct = bad_ct + 1
        bad_files.append(file_paths[0])
print(bad_ct, 1 - bad_ct / (119 + 119))
t2 = time.time()
print('程序运行时间:%s毫秒' % ((t2 - t1)*1000))


# 0.924370
# 0.9243697478991597

# 4152 / 240 = 17.3ms

18 0.9243697478991597
程序运行时间:4152.628421783447毫秒


### onnx2tensorrt

In [1]:
import numpy as np
import tensorrt as trt
from PIL import ImageDraw
import common

import sys, os

# sys.path.insert(1, os.path.join(sys.path[0], ".."))
# from downloader import getFilePath

TRT_LOGGER = trt.Logger()

def get_engine(onnx_file_path, engine_file_path=""):
    """Attempts to load a serialized engine if available, otherwise builds a new TensorRT engine and saves it."""

    def build_engine():
        """Takes an ONNX file and creates a TensorRT engine to run inference with"""
        with trt.Builder(TRT_LOGGER) as builder, builder.create_network(
            common.EXPLICIT_BATCH
        ) as network, builder.create_builder_config() as config, trt.OnnxParser(
            network, TRT_LOGGER
        ) as parser, trt.Runtime(
            TRT_LOGGER
        ) as runtime:
            config.max_workspace_size = 1 << 28  # 256MiB
            builder.max_batch_size = 1
            # Parse model file
            if not os.path.exists(onnx_file_path):
                print(
                    "ONNX file {} not found, please run yolov3_to_onnx.py first to generate it.".format(onnx_file_path)
                )
                exit(0)
            print("Loading ONNX file from path {}...".format(onnx_file_path))
            with open(onnx_file_path, "rb") as model:
                print("Beginning ONNX file parsing")
                if not parser.parse(model.read()):
                    print("ERROR: Failed to parse the ONNX file.")
                    for error in range(parser.num_errors):
                        print(parser.get_error(error))
                    return None
            # The actual yolov3.onnx is generated with batch size 64. Reshape input to batch size 1
            network.get_input(0).shape = [1, 3, 224, 224]
            print("Completed parsing of ONNX file")
            print("Building an engine from file {}; this may take a while...".format(onnx_file_path))
            plan = builder.build_serialized_network(network, config)
            engine = runtime.deserialize_cuda_engine(plan)
            print("Completed creating Engine")
            with open(engine_file_path, "wb") as f:
                f.write(plan)
            return engine

    if os.path.exists(engine_file_path):
        # If a serialized engine exists, use it instead of building an engine.
        print("Reading engine from file {}".format(engine_file_path))
        with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
            return runtime.deserialize_cuda_engine(f.read())
    else:
        return build_engine()

In [2]:
get_engine(r"D:\models\mobilenet_v3_small_20231008_1.onnx", r"D:\models\mobilenet_v3_small_3.trt")

C:\Users\LEGION\AppData\Local\Temp\ipykernel_12216\3092600499.py:25: DeprecationWarning: Use set_memory_pool_limit instead.
  config.max_workspace_size = 1 << 28  # 256MiB
C:\Users\LEGION\AppData\Local\Temp\ipykernel_12216\3092600499.py:26: DeprecationWarning: Use network created with NetworkDefinitionCreationFlag::EXPLICIT_BATCH flag instead.
  builder.max_batch_size = 1


Loading ONNX file from path D:\models\mobilenet_v3_small_20231008_1.onnx...
Beginning ONNX file parsing
Completed parsing of ONNX file
Building an engine from file D:\models\mobilenet_v3_small_20231008_1.onnx; this may take a while...
Completed creating Engine


### tensorrt推理

In [7]:
image_datasets = {x: MyImageDataset(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1, shuffle=False, num_workers=4)  for x in ['train', 'val']}

# Do inference with TensorRT
engine_file = r"D:\models\mobilenet_v3_small_3_f16.trt"
t1 = time.time()
bad_ct = 0
bad_files = []
trt_outputs = []

output_shapes = [(1, 2)]
with get_engine(r"", engine_file) as engine, engine.create_execution_context() as context:
        inputs, outputs, bindings, stream = common.allocate_buffers(engine)
        # Do inference
        
        for image, label, file_paths in dataloaders['val']:
            image = image.numpy()
            inputs[0].host = image
            trt_outputs = common.do_inference_v2(context, bindings=bindings, inputs=inputs, outputs=outputs, stream=stream)
            
            trt_outputs = [output.reshape(shape) for output, shape in zip(trt_outputs, output_shapes)]
            label = label.numpy()
            if np.argmax(trt_outputs[0][0]) != label[0]:
                bad_ct = bad_ct + 1
                bad_files.append(file_paths[0])


print(bad_ct, 1 - bad_ct / (119 + 119))
t2 = time.time()
print('程序运行时间:%s毫秒' % ((t2 - t1)*1000))

# Reading engine from file D:\models\mobilenet_v3_small_3.trt
# 18 0.9243697478991597
# 程序运行时间:3278.118848800659毫秒

# Reading engine from file D:\models\mobilenet_v3_small_3_f16.trt
# 18 0.9243697478991597
# 程序运行时间:3466.071844100952毫秒

Reading engine from file D:\models\mobilenet_v3_small_3_f16.trt
18 0.9243697478991597
程序运行时间:3466.071844100952毫秒


### TorchScript

In [ ]:
inputs, classes = next(iter(dataloaders['train']))
inputs = inputs.to(device)
traced_script_module = torch.jit.trace(model_ft, inputs)
traced_script_module(inputs)
traced_script_module.save(r"C:/Users/LEGION/traced_" + model_name)

### 推理

In [4]:
# model_name = "WYDM_efficientnet_v2_m_20230905_1.pt"
# model_ft.load_state_dict(torch.load(model_name))
# model_ft.eval()

image_datasets = {x: MyImageDataset(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16, shuffle=False, num_workers=4)  for x in ['train', 'val']}

bad_ct = 0
bad_files = []
for inputs, labels, file_paths in dataloaders['val']:
    inputs = inputs.to(device)
    #labels = labels.to(device)
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        #loss = criterion(outputs, labels)
        for p, l, f in zip(preds.to('cpu').numpy(), labels.numpy(), file_paths):
            if p != l:
                bad_ct = bad_ct + 1
                bad_files.append(f)
                print(f)
print(bad_ct, bad_ct / (1048 + 1048))

import shutil
for file in bad_files:
    target_path = file.replace("val", "res")
    shutil.copy(file, target_path)

D:/pictures/wydm/deep learning\val\0\002803_501.png
D:/pictures/wydm/deep learning\val\0\091124_847.png
D:/pictures/wydm/deep learning\val\0\092645_242.png
D:/pictures/wydm/deep learning\val\0\092831_566.png
D:/pictures/wydm/deep learning\val\0\093023_692.png
D:/pictures/wydm/deep learning\val\0\093718_488.png
D:/pictures/wydm/deep learning\val\0\093926_878.png
D:/pictures/wydm/deep learning\val\0\094014_460.png
D:/pictures/wydm/deep learning\val\0\100436_159.png
D:/pictures/wydm/deep learning\val\0\103931_750.png
D:/pictures/wydm/deep learning\val\0\105022_470.png
D:/pictures/wydm/deep learning\val\0\105631_050.png
D:/pictures/wydm/deep learning\val\0\105736_322.png
D:/pictures/wydm/deep learning\val\0\110534_368.png
D:/pictures/wydm/deep learning\val\0\114936_003.png
D:/pictures/wydm/deep learning\val\0\12 (85).png
D:/pictures/wydm/deep learning\val\0\131932_967.png
D:/pictures/wydm/deep learning\val\0\142155_816.png
D:/pictures/wydm/deep learning\val\0\161148_962.png
D:/pictures/wyd

In [3]:
# model_ft.load_state_dict(torch.load("WYDM_efficientnet_v2_m_5000.pt"))
model_ft.load_state_dict(torch.load("WYDM_efficientnet_v2_m_6000_20230909_10.pt"))
model_ft.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [4]:
bad_ct = 0
bad_files = []
for inputs, labels, file_paths in dataloaders['val']:
    inputs = inputs.to(device)
    #labels = labels.to(device)
    with torch.set_grad_enabled(False):
        outputs = model_ft(inputs)
        _, preds = torch.max(outputs, 1)
        #loss = criterion(outputs, labels)
        for p, l, f in zip(preds.to('cpu').numpy(), labels.numpy(), file_paths):
            if p != l:
                bad_ct = bad_ct + 1
                bad_files.append(f)
                print(f)
print(bad_ct, bad_ct / (1048 + 1048))

import shutil
for file in bad_files:
    target_path = file.replace("val", "res")
    shutil.copy(file, target_path)

ValueError: not enough values to unpack (expected 3, got 2)

In [ ]:
bad_files = ['D:/pictures/wydm/deep learning\\val\\0\\x (86).png', 'D:/pictures/wydm/deep learning\\val\\1\\141926_773.png', 'D:/pictures/wydm/deep learning\\val\\1\\173512_837.png', 'D:/pictures/wydm/deep learning\\val\\1\\142211_422.png', 'D:/pictures/wydm/deep learning\\val\\1\\141952_579.png', 'D:/pictures/wydm/deep learning\\val\\1\\164026_881.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (564).png', 'D:/pictures/wydm/deep learning\\val\\1\\182410_994.png', 'D:/pictures/wydm/deep learning\\val\\1\\090743_647.png', 'D:/pictures/wydm/deep learning\\val\\1\\142046_129.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (810).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (932).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (90).png', 'D:/pictures/wydm/deep learning\\val\\1\\57 (13).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (478).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (104).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (479).png', 'D:/pictures/wydm/deep learning\\val\\1\\61 (14).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (995).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (965).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (591).png', 'D:/pictures/wydm/deep learning\\val\\1\\141946_878.png', 'D:/pictures/wydm/deep learning\\val\\1\\213105_294.png', 'D:/pictures/wydm/deep learning\\val\\1\\51 (1).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (108).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (755).png', 'D:/pictures/wydm/deep learning\\val\\1\\092643_194.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (27).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (570).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (786).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (937).png', 'D:/pictures/wydm/deep learning\\val\\1\\142124_808.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (531).png', 'D:/pictures/wydm/deep learning\\val\\1\\23 (4).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (724).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (636).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (527).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (763).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (642).png', 'D:/pictures/wydm/deep learning\\val\\1\\56 (2).png', 'D:/pictures/wydm/deep learning\\val\\1\\57 (89).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (133).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (562).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (1007).png', 'D:/pictures/wydm/deep learning\\val\\1\\195001_475.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (696).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (156).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (910).png', 'D:/pictures/wydm/deep learning\\val\\1\\59_4 (132).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (506).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (630).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (643).png', 'D:/pictures/wydm/deep learning\\val\\1\\105421_260.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (821).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (34).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (131).png', 'D:/pictures/wydm/deep learning\\val\\1\\51 (3).png', 'D:/pictures/wydm/deep learning\\val\\1\\163252_570.png', 'D:/pictures/wydm/deep learning\\val\\1\\Bmp (31).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (935).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (563).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (519).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (738).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (758).png', 'D:/pictures/wydm/deep learning\\val\\1\\092636_403.png', 'D:/pictures/wydm/deep learning\\val\\1\\105038_462.png', 'D:/pictures/wydm/deep learning\\val\\1\\212715_447.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (961).png', 'D:/pictures/wydm/deep learning\\val\\1\\12 (74).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (818).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (580).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (967).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (12).png', 'D:/pictures/wydm/deep learning\\val\\1\\231909_822.png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (1014).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (483).png', 'D:/pictures/wydm/deep learning\\val\\1\\165901_899.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (130).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (748).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (689).png', 'D:/pictures/wydm/deep learning\\val\\1\\b (98).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (725).png', 'D:/pictures/wydm/deep learning\\val\\1\\092635_934.png', 'D:/pictures/wydm/deep learning\\val\\1\\Bmp (40).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (97).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (657).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (79).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (653).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (142).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (514).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (85).png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (754).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (480).png', 'D:/pictures/wydm/deep learning\\val\\1\\180237_838.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (941).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (966).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (165).png', 'D:/pictures/wydm/deep learning\\val\\1\\094149_157.png', 'D:/pictures/wydm/deep learning\\val\\1\\131338_837.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (969).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (532).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (786).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (169).png', 'D:/pictures/wydm/deep learning\\val\\1\\12 (76).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (606).png', 'D:/pictures/wydm/deep learning\\val\\1\\59_4 (205).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (745).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (849).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (595).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (934).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (708).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (625).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (610).png', 'D:/pictures/wydm/deep learning\\val\\1\\57 (113).png', 'D:/pictures/wydm/deep learning\\val\\1\\142028_618.png', 'D:/pictures/wydm/deep learning\\val\\1\\141924_461.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (81).png', 'D:/pictures/wydm/deep learning\\val\\1\\59_4 (106).png', 'D:/pictures/wydm/deep learning\\val\\1\\094143_158.png', 'D:/pictures/wydm/deep learning\\val\\1\\c1 (651).png', 'D:/pictures/wydm/deep learning\\val\\1\\110131_291.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (938).png', 'D:/pictures/wydm/deep learning\\val\\1\\141821_931.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (561).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (782).png', 'D:/pictures/wydm/deep learning\\val\\1\\24 (3).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (944).png', 'D:/pictures/wydm/deep learning\\val\\1\\55 (16).png', 'D:/pictures/wydm/deep learning\\val\\1\\8 (118).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (711).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (555).png', 'D:/pictures/wydm/deep learning\\val\\1\\135133_055.png', 'D:/pictures/wydm/deep learning\\val\\1\\093011_414.png', 'D:/pictures/wydm/deep learning\\val\\0\\x (116).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (632).png', 'D:/pictures/wydm/deep learning\\val\\0\\x (528).png', 'D:/pictures/wydm/deep learning\\val\\1\\c3 (38).png', 'D:/pictures/wydm/deep learning\\val\\1\\094207_134.png', 'D:/pictures/wydm/deep learning\\val\\1\\8 (71).png']
idx = 3
print(bad_files[idx*3 + 0])
print(bad_files[idx*3 + 1])
print(bad_files[idx*3 + 2])
img1 = cv2.cvtColor(cv2.imread(bad_files[idx*3 + 0]), cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(cv2.imread(bad_files[idx*3 + 1]), cv2.COLOR_BGR2RGB)
img3 = cv2.cvtColor(cv2.imread(bad_files[idx*3 + 2]), cv2.COLOR_BGR2RGB)
plot_3_img(img1, img2, img3)